# Troubleshooting, Tips, and Tricks

## Getting Help

* The [#tutorial channel](https://ray-distributed.slack.com/archives/C011ML23W5B) on the [Ray Slack](https://ray-distributed.slack.com). [Click here](https://forms.gle/9TSdDYUgxYs8SA9e8) to join.
* [Email](mailto:academy@anyscale.com)

Find an issue? Please report it!

* [GitHub issues](https://github.com/anyscale/academy/issues)

## For More Information on Ray and Anyscale

* [ray.io](https://ray.io): The Ray website. In particular:
    * [Documentation](https://ray.readthedocs.io/en/latest/): The full Ray documentation
    * [Blog](https://medium.com/distributed-computing-with-ray): The Ray blog
    * [GitHub](https://github.com/ray-project/ray): The source code for Ray
* [anyscale.com](https://anyscale.com/): The company developing Ray and these tutorials. In particular:
    * [Blog](https://anyscale.com/blog/): The Anyscale blog
    * [Events](https://anyscale.com/events/): Online events, [Ray Summit](http://raysummit.org), and meetups   
    * [Academy](https://anyscale.com/academy/): Training for Ray and Anyscale products    
    * [Jobs](https://jobs.lever.co/anyscale): Yes, we're hiring!
* Community:
    * [Ray Slack](ray-distributed.slack.com) ([Click here](https://forms.gle/9TSdDYUgxYs8SA9e8) to join): The best forum for help on Ray. Use the `#tutorials` channel to ask for help on these tutorials!
    * [ray-dev mailing list](https://groups.google.com/forum/?nomobile=true#!forum/ray-dev)
    * [@raydistributed](https://twitter.com/raydistributed)
    * [@anyscalecompute](https://twitter.com/anyscalecompute)

## Troubleshooting

### ray.init() Fails

Suppose you get an error like this:

```
... INFO services.py:... -- Failed to connect to the redis server, retrying.
```

It probably means you are running a VPN on your machine. [At this time](https://github.com/ray-project/ray/issues/6573), you can't use `ray.init()` with a VPN running. You'll have to stop your VPN to run `ray.init()`, then once it finishes, you can restart your VPN.

Suppose you successfully executed `!../tools/start-ray.sh` in a notebook, but then when you run `ray.init(adress='auto', ...)` you get the following error for some `IP` address and `PORT`:

``` 
ConnectionError: Error 61 connecting to IP:PORT. Connection refused.
```

If the output of `../tools/start-ray.sh` includes instructions for passing a Redis password, e.g., `redis_password='5241590000000000'`, add this argument to the `ray.init()` call and try again. 

If that fails, it may be necessary to kill any old Redis processes that are running. On MacOS and Linux systems, use a terminal window and run the following command, shown with example output:

```shell
$ ps -ef | grep redis
501 36029     1   0  1:53PM ??         0:00.03 .../lib/python3.7/site-packages/ray/core/src/ray/thirdparty/redis/src/redis-server *:48044
501 36030     1   0  1:53PM ??         0:00.02 .../lib/python3.7/site-packages/ray/core/src/ray/thirdparty/redis/src/redis-server *:42902

$ kill 36029 36039
```

Then try again.

### MacOS Annoyances

If `ray.init()` worked (for example, you see a message like _View the Ray dashboard at localhost:8265_) and you're using a Mac, you may get several annoying dialogs asking you if you want to allow incoming connections for `python` and/or `redis-server`. Click "Accept" for each one and they shouldn't appear again during this lesson. MacOS is trying to verify if these executables have been properly signed. Ray uses Redis. If you installed Python using Anaconda or other mechanism, then it probably isn't properly signed from the point of view of MacOS. To permanently fix this problem, [see this StackExchange post](https://apple.stackexchange.com/questions/3271/how-to-get-rid-of-firewall-accept-incoming-connections-dialog).

### "File has changed on disk"

On rare occasions, you'll get a warning that the notebook file you are viewing has change on disk. Usually, you'll just want to keep the file you're viewing in Jupyter, especially if you have already made changes, but reloading it is fine, too.

### "AssertionError" While Graphs Are Being Drawn 

We've observed a few occasions when errors like the following are written in the notebook outputs (very long numbers truncated):

```
WARNING:param.dynamic_operation: Callable raised "AssertionError('New option id 43948... does not match any option trees in Store.custom_options.',)".

AssertionError: New option id 11997... does not match any option trees in Store.custom_options.
```

They are related to the graphing libraries we use and appear to be harmless.

### "ImportError: No module named gym"

If you are working on a laptop and using Anaconda, and you get import errors for modules like `gym`, e.g., _ImportError: No module named gym_, this probably indicates you have the wrong `python` and `pip` commands on your path ahead of the versions for your Conda environment. To verify, use the `which` command:

```shell
$ which -a python
/usr/bin/python
/Users/me/anaconda3/envs/anyscale-academy/bin/python
```

In this example, the built-in `python` is ahead of the correct Conda environment `python`.

To fix, try the following:

First, try activiting the `anyscale-academy` environment again: 

```shell
conda activate anyscale-academy
```

(see the [README](../README.md) for more details about the Conda environment we are using).

Run `which -a python` again. Does it show your `~/anaconda3/envs/anyscale-academy/bin/python` first? If not, force the correct ordering by changing the path: 

```shell
export PATH=$HOME/anaconda3/envs/anyscale-academy/bin:$PATH
```

Run the `which` command again to confirm the correct `python` is now first in the list.

### Saving Resources by Ending Sessions You No Longer Need

When doing lessons on a machine or cloud instance with low resources, close notebooks you no longer need and stop their kernels. Do the latter by clicking the left-hand icon with a white square inside a black circle. The tool tip says _Running Terminals and Kernels_. Click _SHUT DOWN_ for any notebooks you no longer need.

## Tips and Tricks

### Jupyter Tips

* **Learning Jupyter Lab:** Open _Help > JupyerLab Reference_ for a user guide and detailed information about Jupyter Lab. The _Help_ menu has other references, too.
* **Excessive Output:** If the output of a cell is very long, right click on it and select _Enable Scrolling for Outputs_.

### General Task and Actor Tips

* To create a task from a function or an actor from a class, annotate it with `@ray.remote`.
* Invoke tasks with `foo.remote(...)`
* Invoke actor constructors `MyClass.remote(...)`.
* Invoke actor methods with `my_instance.remote(...)`.
* Invocations return an `ObjectID` for a _future_. Use `ray.get(id)` to return the value.
* However, `ray.get()` blocks, so consider using `ray.wait()` when waiting for a collection of futures, so as they become available, you can retrieve them with `ray.get()` (which won't block when we know the results are available), then process the results, all while waiting for the rest to finish.
* Pick functions to parallelize that do enough work so that the Ray "remoting" overhead is not significant. Very short functions will actually yield lower performance if convert to tasks. 
* Similarly, avoid too many actors, as each one is pinned to memory until no longer needed.

#### Using Existing Functions and Classes

An existing function can be used as a task by defining a new task function that calls the original function. For example:

```python
def work(args):
    do_work(...)
    
@ray.remote
def remote_work(args):
    do_work(args)
```

This allows you to use either version, as appropriate.

Similarly, existing classes can be subclassed to create actors:

```python
class Counter():
    def __init__(self, init_count):
        self.count = init_count
        
    def increment():
        self.count +=1
        return self.count
    
@ray.remote
class RemoteCounter(Counter):
    def __init__(self, init_count):
        super().__init__(init_count)
        
    def get_count():
        return self.count
```

Note that we added a `get_count()` method, because member attributes can't be accessed directly, in contrast with normal classes. Of course, you don't need to add _accessor_ methods unless you need them.

### Using the Ray Dashboard

#### Opening the Dashboard

As it executes, `ray.init` prints the dashboard URL.

You can get the URL later if needed using `ray.get_webui_url()`.

> **Tip:** In a notebook, use a cell with `f'http://{ray.get_webui_url()}'`. The full URL printed will be clickable.

#### Profiling Actors

The _Logical View_ offers a powerful and convenient way to profile actor performance using [flame graphs](http://www.brendangregg.com/flamegraphs.html). Details are in the [Dashboard docs](https://ray.readthedocs.io/en/latest/ray-dashboard.html#ray-dashboard).

This feature uses [py-spy](https://github.com/benfred/py-spy) to instrument and profile the application. Unfortunately, you may be asked to enter the `sudo` password to use this feature, because of the way it instruments processes. Currently, the only way to get this to work with the Dashboard launched from notebooks is to use _passwordless sudo_. On MacOS and Linux systems, it should be sufficient to add a line like the following the `/etc/sudoers` (edited using `sudo visudo`!):

```
yourusername ALL = (ALL) NOPASSWD: ALL
```

Carefully the consider the security implications of this change!!

The alternative is to run your Python application from a command line, then open the Dashboard. When you click a link to profile code, as discussed next, you'll be prompted for your password in the terminal. (The prompt could get mixed with other output from the program.) Enter your password there and the profiling will continue.

Using either approach, to profile with the Dashboard, click the _Logical View_ tab. It shows a list of actors that have been run or are running. Find the running actor that appears to be the one you want to profile. You'll see a line like this:

> Actor <hex_number> (Profile for 10s 30s 60s) Kill Actor

The _10s, 30s, 60s_ are links. Click the time you want (pick one that's shorter than the remaining execution time). 

When it finishes, click _Profile results_. A new tab opens with the _speedscope_ view of the data, which shows a flame graph. You can learn more about navigating and using this tool at the [speedscope GitHub site](https://github.com/jlfwong/speedscope).

A lot of the data will be related to actor messaging and not normally interesting. Look at the following screen shot, cropped from a screen shot in [lesson 4]():

![Conway's GoL Flame Graph](../images/ConwaysGameOfLife-FlameGraph-crop.png)

Note the arrow and pagination on the upper-right hand side. Sometimes navigating pages will take you to interesting data. Also, the _Left Heavy_ button on the upper-left hand side is clicked, so the view is zoomed into the interesting data about the `step()` method profiled here.

### Profiling Code with ray.timeline()

The other built-in way to profile performance uses `ray.timeline(file)` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.timeline)). It requires a Chrome web browser to view the data. This is the only way to profile tasks.

Use it as follows:

```
ray.timeline('timeline.txt')
my_long_task.remote(...)  # task to profile
```

Then, open chrome://tracing in the Chrome web browser (only Chrome is supported) and click the _load_ button to load the file. To zoom in or out, click the asymmetric up-down arrow button. To move around, click the crossed arrow and drag a section in view. Click on a box in the timeline to see details about it. 

Look for blocks corresponding to long-running tasks and look for idle periods, which reflect processing outside the context of Ray.

### Cleaning Up

When your driver script exists, it will implicitly call `ray.shutdown()`. This can be useful to do explicitly if you want to restart Ray, for example, in a notebook environment, like this:

```python
ray.shutdown()
ray.init(...)
```